In [35]:
# 간단한 테스트 스크립트
from qdrant_client import QdrantClient
from config.settings import QDRANT_PATH, QDRANT_COLLECTION

client = QdrantClient(path=QDRANT_PATH)

# 이미지가 있는 청크 찾기
results = client.scroll(
    collection_name=QDRANT_COLLECTION,
    limit=10,
    with_payload=True
)

for point in results[0]:
    if point.payload.get("has_image"):
        print(f"\n📄 {point.payload['page_title']}")
        print(f"🖼️ 이미지 개수: {len(point.payload['image_paths'])}")
        print(f"📝 원본 텍스트: {point.payload['text'][:100]}...")
        print(f"🔍 검색용 텍스트: {point.payload['combined_text'][:100]}...")
        print(f"💬 이미지 설명: {point.payload['image_descriptions']}")


📄 테니스 기본 규칙 및 모멘텀 예측 논문 리뷰
🖼️ 이미지 개수: 2
📝 원본 텍스트: ## HMM (Hidden Markov Model)
• HMM = 시간에 따른 숨겨진 상태 전이 + 관측값 발생
• 상태는 시간 순서에 따라 이전 상태 → 현재 상태로 전이 (Ma...
🔍 검색용 텍스트: ## HMM (Hidden Markov Model)
• HMM = 시간에 따른 숨겨진 상태 전이 + 관측값 발생
• 상태는 시간 순서에 따라 이전 상태 → 현재 상태로 전이 (Ma...
💬 이미지 설명: ["[이미지 로드 실패: [Errno 2] No such file or directory: 'notion_images//home/work/rag/Project/rag-report-generator/data/notion_images/23da296c-8853-8075-afd1-ca7daa3c3e7f_23da296c-8853-80b2-9e07-c5d1a36f24ce.png']", "[이미지 로드 실패: [Errno 2] No such file or directory: 'notion_images//home/work/rag/Project/rag-report-generator/data/notion_images/23da296c-8853-8075-afd1-ca7daa3c3e7f_23da296c-8853-8089-8f51-f7ab123680cd.png']"]

📄 테니스 기본 규칙 및 모멘텀 예측 논문 리뷰
🖼️ 이미지 개수: 1
📝 원본 텍스트: # 논문 리뷰 - Capturing Momentum: Tennis Match Analysis Using Machine Learning and Time Series Theory
[I...
🔍 검색용 텍스트: # 논문 리뷰 - Capturing Momentum: Tennis Match Analysis Using Machine Learning and Time Series Theory
[이...
💬 이미지 설명: ["[이미지 로드 실패: [Errn

In [2]:
import os
import re
import json
import hashlib
import base64
import httpx
from datetime import datetime
from typing import Optional, List, Dict, Any
from dataclasses import dataclass, field
from dotenv import load_dotenv

from notion_client import Client
from openai import OpenAI
from qdrant_client import QdrantClient
from qdrant_client.models import (
    VectorParams, Distance, PointStruct,
    Filter, FieldCondition, MatchValue,
    PayloadSchemaType
)
from utils.prompt_loader import load_prompt

load_dotenv()


True

In [32]:

# ========== 설정 ==========
NOTION_TOKEN = os.environ.get("NOTION_TOKEN")
DATA_SOURCE_ID = os.environ.get("DATA_SOURCE_ID")  # 필수!
IMAGE_DIR = "notion_images"

OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY")
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"

AZURE_AI_CREDENTIAL = os.environ.get("AZURE_AI_CREDENTIAL")
AZURE_AI_ENDPOINT = os.environ.get("AZURE_AI_ENDPOINT", "https://models.inference.ai.azure.com")

EMBEDDING_MODEL = "openai/text-embedding-3-large"
EMBEDDING_DIM = 3072
IMAGE_MODEL = "gpt-4.1"

CHUNK_SIZE = 800
CHUNK_OVERLAP = 50
IMAGE_CONTEXT_CHARS = 300
QDRANT_COLLECTION = "notion_docs"



# ========== 데이터 클래스 ==========
@dataclass
class Chunk:
    chunk_id: str
    page_id: str
    text: str
    combined_text: str = ""
    has_image: bool = False
    image_paths: list = field(default_factory=list)  # ✅ 추가
    image_descriptions: list = field(default_factory=list)  # ✅ 추가
    page_title: str = ""
    section_title: str = ""
    section_path: list = field(default_factory=list)
    heading_level: int = 0
    chunk_index: int = 0
    created_time: str = ""
    last_edited_time: str = ""
    properties: dict = field(default_factory=dict)

from datetime import datetime
from typing import List, Dict, Set

# ========== 데이터 존재 여부 확인 ==========
def check_existing_data(filepath: str = "notion_data.json") -> Dict:
    """
    기존 데이터 존재 여부 및 통계 확인
    
    Returns:
        dict: {
            "exists": bool,
            "count": int,
            "page_ids": set,
            "last_updated": str
        }
    """
    if not os.path.exists(filepath):
        print(f"❌ 파일 없음: {filepath}")
        return {
            "exists": False,
            "count": 0,
            "page_ids": set(),
            "last_updated": None
        }
    
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        page_ids = {item["page_id"] for item in data}
        
        # 마지막 업데이트 시간 찾기
        last_times = [item.get("last_edited_time", "") for item in data]
        last_updated = max(last_times) if last_times else None
        
        print(f"✅ 기존 데이터: {len(data)}개 페이지")
        print(f"   마지막 업데이트: {last_updated}")
        
        return {
            "exists": True,
            "count": len(data),
            "page_ids": page_ids,
            "last_updated": last_updated
        }
    except Exception as e:
        print(f"⚠️ 파일 로드 실패: {e}")
        return {
            "exists": False,
            "count": 0,
            "page_ids": set(),
            "last_updated": None
        }


def check_qdrant_data(qdrant_client: QdrantClient) -> Dict:
    """
    Qdrant 컬렉션의 데이터 확인
    
    Returns:
        dict: {
            "exists": bool,
            "count": int,
            "page_ids": set
        }
    """
    try:
        collections = qdrant_client.get_collections().collections
        exists = any(c.name == QDRANT_COLLECTION for c in collections)
        
        if not exists:
            print(f"❌ Qdrant 컬렉션 없음: {QDRANT_COLLECTION}")
            return {"exists": False, "count": 0, "page_ids": set()}
        
        # 컬렉션 정보 가져오기
        info = qdrant_client.get_collection(QDRANT_COLLECTION)
        count = info.points_count
        
        # 모든 page_id 추출 (샘플링)
        scroll_result = qdrant_client.scroll(
            collection_name=QDRANT_COLLECTION,
            limit=10000,
            with_payload=["page_id"]
        )
        
        page_ids = {point.payload["page_id"] for point in scroll_result[0]}
        
        print(f"✅ Qdrant 데이터: {count}개 청크, {len(page_ids)}개 페이지")
        
        return {
            "exists": True,
            "count": count,
            "page_ids": page_ids
        }
    except Exception as e:
        print(f"⚠️ Qdrant 확인 실패: {e}")
        return {"exists": False, "count": 0, "page_ids": set()}


# ========== 증분 수집 ==========
def collect_missing_pages(
    collector: NotionDataSourceCollector,
    existing_page_ids: Set[str],
    filepath: str = "notion_data.json"
) -> List[Dict]:
    """
    기존 데이터에 없는 새 페이지만 수집
    
    Args:
        collector: NotionDataSourceCollector 인스턴스
        existing_page_ids: 이미 수집된 page_id 집합
        filepath: 저장할 파일 경로
    
    Returns:
        새로 수집한 페이지 데이터 리스트
    """
    print("\n🔍 Notion에서 전체 페이지 목록 가져오는 중...")
    all_pages = collector.get_all_pages_from_datasource()
    
    # 새 페이지 필터링
    new_pages = [p for p in all_pages if p["id"] not in existing_page_ids]
    
    print(f"\n📊 분석 결과:")
    print(f"   전체 페이지: {len(all_pages)}개")
    print(f"   기존 페이지: {len(existing_page_ids)}개")
    print(f"   새 페이지: {len(new_pages)}개")
    
    if not new_pages:
        print("\n✅ 모든 페이지가 이미 수집되어 있습니다!")
        return []
    
    print(f"\n🚀 {len(new_pages)}개 새 페이지 수집 시작...\n")
    
    new_data = []
    for idx, page in enumerate(new_pages):
        page_id = page["id"]
        properties = collector.extract_page_properties(page)
        title = collector.get_page_title(properties)
        
        print(f"[{idx+1}/{len(new_pages)}] 📄 {title}")
        
        try:
            blocks = collector.get_all_blocks(page_id)
            content_lines = [collector.extract_block_content(b, page_id) for b in blocks]
            full_content = "\n".join(filter(None, content_lines))
            
            new_data.append({
                "page_id": page_id,
                "title": title,
                "created_time": page.get("created_time", ""),
                "last_edited_time": page.get("last_edited_time", ""),
                "properties": properties,
                "content": full_content
            })
            print(f"  → {len(blocks)}개 블록, {len(full_content)}자")
        except Exception as e:
            print(f"  ⚠️ 실패: {e}")
            new_data.append({
                "page_id": page_id, 
                "title": title, 
                "content": "",
                "created_time": page.get("created_time", ""),
                "last_edited_time": page.get("last_edited_time", ""),
                "properties": properties
            })
    
    # 기존 데이터와 병합
    if os.path.exists(filepath):
        existing_data = load_data(filepath)
        merged_data = existing_data + new_data
        save_data(merged_data, filepath)
        print(f"\n💾 병합 완료: {len(existing_data)} + {len(new_data)} = {len(merged_data)}개")
    else:
        save_data(new_data, filepath)
        print(f"\n💾 새 파일 생성: {len(new_data)}개")
    
    return new_data


def update_changed_pages(
    collector: NotionDataSourceCollector,
    existing_data: List[Dict],
    filepath: str = "notion_data.json"
) -> List[Dict]:
    """
    수정된 페이지 업데이트
    
    Args:
        collector: NotionDataSourceCollector 인스턴스
        existing_data: 기존 데이터
        filepath: 저장할 파일 경로
    
    Returns:
        업데이트된 전체 데이터
    """
    print("\n🔄 수정된 페이지 확인 중...")
    
    all_pages = collector.get_all_pages_from_datasource()
    page_map = {p["id"]: p for p in all_pages}
    
    updated_data = []
    update_count = 0
    
    for old_item in existing_data:
        page_id = old_item["page_id"]
        
        if page_id not in page_map:
            # Notion에서 삭제된 페이지
            print(f"  ⚠️ 삭제된 페이지: {old_item['title']}")
            continue
        
        new_page = page_map[page_id]
        old_time = old_item.get("last_edited_time", "")
        new_time = new_page.get("last_edited_time", "")
        
        # 수정 시간 비교
        if new_time > old_time:
            print(f"  🔄 업데이트: {old_item['title']}")
            
            properties = collector.extract_page_properties(new_page)
            title = collector.get_page_title(properties)
            
            try:
                blocks = collector.get_all_blocks(page_id)
                content_lines = [collector.extract_block_content(b, page_id) for b in blocks]
                full_content = "\n".join(filter(None, content_lines))
                
                updated_data.append({
                    "page_id": page_id,
                    "title": title,
                    "created_time": new_page.get("created_time", ""),
                    "last_edited_time": new_time,
                    "properties": properties,
                    "content": full_content
                })
                update_count += 1
            except Exception as e:
                print(f"    ⚠️ 업데이트 실패: {e}")
                updated_data.append(old_item)
        else:
            updated_data.append(old_item)
    
    if update_count > 0:
        save_data(updated_data, filepath)
        print(f"\n✅ {update_count}개 페이지 업데이트 완료")
    else:
        print("\n✅ 수정된 페이지 없음")
    
    return updated_data


# ========== 전체 파이프라인 ==========
def smart_data_collection(
    force_recreate: bool = False,
    check_updates: bool = True,
    filepath: str = "notion_data.json"
) -> List[Dict]:
    """
    스마트 데이터 수집 파이프라인
    
    Args:
        force_recreate: True면 전체 재수집
        check_updates: True면 수정된 페이지도 확인
        filepath: 데이터 파일 경로
    
    Returns:
        최종 데이터 리스트
    """
    print("=" * 60)
    print("🤖 Notion 스마트 데이터 수집 시작")
    print("=" * 60)
    
    collector = NotionDataSourceCollector(NOTION_TOKEN, DATA_SOURCE_ID)
    
    if force_recreate:
        print("\n♻️ 전체 재수집 모드")
        if os.path.exists(filepath):
            backup = f"{filepath}.backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
            os.rename(filepath, backup)
            print(f"   백업: {backup}")
        
        all_data = collector.collect_all()
        save_data(all_data, filepath)
        return all_data
    
    # 기존 데이터 확인
    existing_info = check_existing_data(filepath)
    
    if not existing_info["exists"]:
        print("\n📥 초기 수집 시작...")
        all_data = collector.collect_all()
        save_data(all_data, filepath)
        return all_data
    
    # 새 페이지 수집
    new_data = collect_missing_pages(collector, existing_info["page_ids"], filepath)
    
    # 수정된 페이지 업데이트
    if check_updates and new_data:
        print("\n" + "=" * 60)
        all_data = load_data(filepath)
        all_data = update_changed_pages(collector, all_data, filepath)
    else:
        all_data = load_data(filepath)
    
    print("\n" + "=" * 60)
    print(f"🎉 완료! 총 {len(all_data)}개 페이지")
    print("=" * 60)
    
    return all_data


def smart_indexing(
    openai_client: OpenAI,
    qdrant_client: QdrantClient,
    image_client: OpenAI = None,
    force_recreate: bool = False,
    data_filepath: str = "notion_data.json"
):
    """
    스마트 인덱싱 파이프라인
    
    Args:
        openai_client: 임베딩용 OpenAI 클라이언트
        qdrant_client: Qdrant 클라이언트
        image_client: 이미지 설명 생성용 클라이언트
        force_recreate: True면 Qdrant 컬렉션 재생성
        data_filepath: 데이터 파일 경로
    """
    print("\n" + "=" * 60)
    print("🔄 Qdrant 인덱싱 시작")
    print("=" * 60)
    
    # Qdrant 상태 확인
    qdrant_info = check_qdrant_data(qdrant_client)
    data_info = check_existing_data(data_filepath)
    
    if not data_info["exists"]:
        print("❌ 데이터 파일이 없습니다. 먼저 데이터를 수집하세요.")
        return
    
    # 새로 인덱싱할 페이지 확인
    if not force_recreate and qdrant_info["exists"]:
        new_page_ids = data_info["page_ids"] - qdrant_info["page_ids"]
        if not new_page_ids:
            print("✅ 모든 데이터가 이미 인덱싱되어 있습니다!")
            return
        
        print(f"\n📊 {len(new_page_ids)}개 새 페이지를 인덱싱합니다.")
        
        # 새 페이지만 로드
        all_data = load_data(data_filepath)
        new_data = [d for d in all_data if d["page_id"] in new_page_ids]
    else:
        init_qdrant(qdrant_client, recreate=force_recreate)
        new_data = load_data(data_filepath)
    
    # 청킹 및 임베딩
    print(f"\n📝 {len(new_data)}개 페이지 처리 중...")
    all_chunks = []
    for page_data in new_data:
        chunks = process_page_data(page_data, openai_client, image_client)
        all_chunks.extend(chunks)
    
    print(f"✅ 총 {len(all_chunks)}개 청크 생성")
    
    if all_chunks:
        embeddings = generate_embeddings(all_chunks, openai_client)
        store_to_qdrant(all_chunks, embeddings, qdrant_client)
    
    print("\n" + "=" * 60)
    print("🎉 인덱싱 완료!")
    print("=" * 60)


# ========== Notion 데이터 수집 (Data Source 전용) ==========
class NotionDataSourceCollector:
    """Notion Data Source에서만 데이터 수집"""
    
    def __init__(self, token: str, data_source_id: str):
        if not data_source_id:
            raise ValueError("DATA_SOURCE_ID가 필요합니다!")
        self.client = Client(auth=token, notion_version=os.environ.get("NOTION_VERSION", "2025-09-03"))
        self.data_source_id = data_source_id
        os.makedirs(IMAGE_DIR, exist_ok=True)
    
    def get_all_blocks(self, block_id: str) -> List[Dict]:
        """모든 블록을 재귀적으로 가져오기 (페이지네이션 포함)"""
        all_blocks = []
        cursor = None
        
        while True:
            response = self.client.blocks.children.list(
                block_id=block_id,
                start_cursor=cursor
            )
            all_blocks.extend(response["results"])
            
            if not response.get("has_more"):
                break
            cursor = response["next_cursor"]
        
        # 하위 블록 재귀 탐색
        for block in all_blocks:
            if block.get("has_children"):
                block["children"] = self.get_all_blocks(block["id"])
        
        return all_blocks
    
    def extract_rich_text(self, rich_text_list: List) -> str:
        """rich_text 배열에서 텍스트 추출"""
        if not rich_text_list:
            return ""
        return "".join([t.get("plain_text", "") for t in rich_text_list])
    
    def download_image(self, url: str, page_id: str, block_id: str) -> Optional[str]:
        """이미지 다운로드 후 로컬 경로 반환"""
        try:
            response = httpx.get(url, timeout=30)
            if response.status_code == 200:
                ext = "png"
                if "." in url.split("?")[0]:
                    ext = url.split("?")[0].split(".")[-1][:4]
                
                filename = f"{IMAGE_DIR}/{page_id}_{block_id}.{ext}"
                with open(filename, "wb") as f:
                    f.write(response.content)
                return filename
        except Exception as e:
            print(f"  ⚠️ 이미지 다운로드 실패: {e}")
        return None
    
    def extract_block_content(self, block: Dict, page_id: str, depth: int = 0) -> str:
        """블록에서 모든 내용 추출"""
        block_type = block["type"]
        block_id = block["id"]
        indent = "  " * depth
        result = ""
        
        # 텍스트 블록들
        text_types = [
            "paragraph", "heading_1", "heading_2", "heading_3",
            "bulleted_list_item", "numbered_list_item", 
            "quote", "toggle", "to_do", "callout"
        ]
        
        if block_type in text_types:
            text = self.extract_rich_text(block[block_type].get("rich_text", []))
            
            if block_type == "heading_1":
                result = f"{indent}# {text}"
            elif block_type == "heading_2":
                result = f"{indent}## {text}"
            elif block_type == "heading_3":
                result = f"{indent}### {text}"
            elif block_type == "bulleted_list_item":
                result = f"{indent}• {text}"
            elif block_type == "numbered_list_item":
                result = f"{indent}1. {text}"
            elif block_type == "quote":
                result = f"{indent}> {text}"
            elif block_type == "to_do":
                checked = "✅" if block["to_do"].get("checked") else "⬜"
                result = f"{indent}{checked} {text}"
            elif block_type == "callout":
                emoji = block["callout"].get("icon", {}).get("emoji", "💡")
                result = f"{indent}{emoji} {text}"
            else:
                result = f"{indent}{text}"
        
        # 코드 블록
        elif block_type == "code":
            code = self.extract_rich_text(block["code"].get("rich_text", []))
            lang = block["code"].get("language", "")
            caption = self.extract_rich_text(block["code"].get("caption", []))
            result = f"{indent}```{lang}\n{code}\n{indent}```"
            if caption:
                result += f"\n{indent}Caption: {caption}"
        
        # 이미지
        elif block_type == "image":
            image_data = block["image"]
            url = None
            if image_data["type"] == "file":
                url = image_data["file"]["url"]
            elif image_data["type"] == "external":
                url = image_data["external"]["url"]
            
            if url:
                local_path = self.download_image(url, page_id, block_id)
                caption = self.extract_rich_text(image_data.get("caption", []))
                result = f"{indent}[Image: {local_path or url}]"
                if caption:
                    result += f" - {caption}"
        
        # 비디오
        elif block_type == "video":
            video_data = block["video"]
            url = video_data.get("file", {}).get("url") or video_data.get("external", {}).get("url", "unknown")
            result = f"{indent}[Video: {url}]"
        
        # 파일
        elif block_type == "file":
            file_data = block["file"]
            url = file_data.get("file", {}).get("url") or file_data.get("external", {}).get("url", "unknown")
            name = file_data.get("name", "file")
            result = f"{indent}[File: {name} - {url}]"
        
        # 북마크
        elif block_type == "bookmark":
            url = block["bookmark"].get("url", "")
            caption = self.extract_rich_text(block["bookmark"].get("caption", []))
            result = f"{indent}[Bookmark: {url}]"
            if caption:
                result += f" - {caption}"
        
        # 임베드
        elif block_type == "embed":
            url = block["embed"].get("url", "")
            result = f"{indent}[Embed: {url}]"
        
        # 테이블 행
        elif block_type == "table_row":
            cells = block["table_row"].get("cells", [])
            row_data = [self.extract_rich_text(cell) for cell in cells]
            result = f"{indent}| " + " | ".join(row_data) + " |"
        
        # 구분선
        elif block_type == "divider":
            result = f"{indent}---"
        
        # 링크 프리뷰
        elif block_type == "link_preview":
            url = block["link_preview"].get("url", "")
            result = f"{indent}[Link: {url}]"
        
        # PDF
        elif block_type == "pdf":
            pdf_data = block["pdf"]
            url = pdf_data.get("file", {}).get("url") or pdf_data.get("external", {}).get("url", "unknown")
            result = f"{indent}[PDF: {url}]"
        
        # 수식
        elif block_type == "equation":
            expr = block["equation"].get("expression", "")
            result = f"{indent}[Equation: {expr}]"
        
        # 자식 페이지
        elif block_type == "child_page":
            title = block["child_page"].get("title", "")
            result = f"{indent}📄 [{title}]"
        
        # 자식 데이터베이스
        elif block_type == "child_database":
            title = block["child_database"].get("title", "")
            result = f"{indent}🗃️ [{title}]"
        
        # 기타 (table, column_list 등은 무시)
        elif block_type not in ["table", "column_list", "column", "synced_block"]:
            result = f"{indent}[{block_type}]"
        
        # 하위 블록 처리
        if "children" in block:
            for child in block["children"]:
                child_content = self.extract_block_content(child, page_id, depth + 1)
                if child_content:
                    result += "\n" + child_content
        
        return result
    
    def extract_page_properties(self, page: Dict) -> Dict:
        """페이지 속성(메타데이터) 추출"""
        props = page.get("properties", {})
        extracted = {}
        
        for name, prop in props.items():
            prop_type = prop["type"]
            try:
                if prop_type == "title":
                    extracted[name] = self.extract_rich_text(prop["title"])
                elif prop_type == "rich_text":
                    extracted[name] = self.extract_rich_text(prop["rich_text"])
                elif prop_type == "number":
                    extracted[name] = prop["number"]
                elif prop_type == "select":
                    extracted[name] = prop["select"]["name"] if prop["select"] else None
                elif prop_type == "multi_select":
                    extracted[name] = [s["name"] for s in prop["multi_select"]]
                elif prop_type == "date":
                    if prop["date"]:
                        extracted[name] = {"start": prop["date"].get("start"), "end": prop["date"].get("end")}
                elif prop_type == "checkbox":
                    extracted[name] = prop["checkbox"]
                elif prop_type == "url":
                    extracted[name] = prop["url"]
                elif prop_type == "email":
                    extracted[name] = prop["email"]
                elif prop_type == "phone_number":
                    extracted[name] = prop["phone_number"]
                elif prop_type == "created_time":
                    extracted[name] = prop["created_time"]
                elif prop_type == "last_edited_time":
                    extracted[name] = prop["last_edited_time"]
                elif prop_type == "created_by":
                    extracted[name] = prop["created_by"].get("name", prop["created_by"].get("id"))
                elif prop_type == "last_edited_by":
                    extracted[name] = prop["last_edited_by"].get("name", prop["last_edited_by"].get("id"))
                elif prop_type == "files":
                    extracted[name] = []
                    for f in prop["files"]:
                        if f["type"] == "file":
                            extracted[name].append(f["file"]["url"])
                        elif f["type"] == "external":
                            extracted[name].append(f["external"]["url"])
                elif prop_type == "relation":
                    extracted[name] = [r["id"] for r in prop["relation"]]
                elif prop_type == "formula":
                    formula = prop["formula"]
                    extracted[name] = formula.get(formula["type"])
                elif prop_type == "rollup":
                    rollup = prop["rollup"]
                    extracted[name] = rollup.get(rollup["type"])
                elif prop_type == "status":
                    extracted[name] = prop["status"]["name"] if prop["status"] else None
                else:
                    extracted[name] = f"[{prop_type}]"
            except Exception:
                extracted[name] = None
        
        return extracted
    
    def get_page_title(self, properties: Dict) -> str:
        """속성에서 제목 추출"""
        for name, value in properties.items():
            if isinstance(value, str) and value and name.lower() in ["title", "name", "이름", "제목"]:
                return value
        for name, value in properties.items():
            if isinstance(value, str) and value:
                return value
        return "Untitled"
    
    def get_all_pages_from_datasource(self) -> List[Dict]:
        """Data Source API로 페이지 가져오기 (페이지네이션 포함)"""
        all_pages = []
        cursor = None
        
        while True:
            response = self.client.data_sources.query(
                data_source_id=self.data_source_id,
                start_cursor=cursor,
                page_size=100
            )
            all_pages.extend(response["results"])
            print(f"  📄 {len(all_pages)}개 페이지 로드됨...")
            
            if not response.get("has_more"):
                break
            cursor = response["next_cursor"]
        
        return all_pages
    
    def collect_all(self, limit: int = None) -> List[Dict]:
        """Data Source의 모든 Notion 데이터 수집"""
        print("🚀 Notion 데이터 수집 시작")
        print(f"📊 Data Source ID: {self.data_source_id}\n")
        
        pages = self.get_all_pages_from_datasource()
        print(f"\n✅ 총 {len(pages)}개 페이지 발견\n")

        if limit:
            pages = pages[:limit]   # ✅ 여기서 제한     

        all_data = []
        for idx, page in enumerate(pages):
            page_id = page["id"]
            properties = self.extract_page_properties(page)
            title = self.get_page_title(properties)
            
            print(f"[{idx+1}/{len(pages)}] 📄 {title}")
            
            try:
                blocks = self.get_all_blocks(page_id)
                content_lines = [self.extract_block_content(b, page_id) for b in blocks]
                full_content = "\n".join(filter(None, content_lines))
                
                all_data.append({
                    "page_id": page_id,
                    "title": title,
                    "created_time": page.get("created_time", ""),
                    "last_edited_time": page.get("last_edited_time", ""),
                    "properties": properties,
                    "content": full_content
                })
                print(f"  → {len(blocks)}개 블록, {len(full_content)}자")
            except Exception as e:
                print(f"  ⚠️ 실패: {e}")
                all_data.append({
                    "page_id": page_id, 
                    "title": title, 
                    "content": "",
                    "created_time": page.get("created_time", ""),
                    "last_edited_time": page.get("last_edited_time", ""),
                    "properties": properties
                })
        
        print(f"\n🎉 완료! 총 {len(all_data)}개 페이지 수집")
        return all_data


# ========== 청킹 함수들 ==========
def split_by_markdown_headers(content: str, page_id: str, page_title: str) -> List[Dict]:
    """마크다운 헤더 기준으로 섹션 분리"""
    header_pattern = r'^(#{1,3})\s+(.+)$'
    lines = content.split('\n')
    sections = []
    current = {"title": page_title, "level": 0, "path": [page_title], "content": []}
    
    for line in lines:
        match = re.match(header_pattern, line)
        if match:
            if current["content"] or current["title"]:
                sections.append(current.copy())
            
            level = len(match.group(1))
            title = match.group(2).strip()
            path = [title] if level == 1 else [page_title, title]
            current = {"title": title, "level": level, "path": path, "content": []}
        else:
            current["content"].append(line)
    
    if current["content"] or current["title"]:
        sections.append(current)
    
    result = []
    for sec in sections:
        content_text = '\n'.join(sec["content"]).strip()
        if content_text or sec["level"] > 0:
            full_text = f"{'#' * sec['level']} {sec['title']}\n{content_text}".strip() if sec["level"] > 0 else content_text
            result.append({"title": sec["title"], "level": sec["level"], "path": sec["path"], "full_text": full_text})
    
    return result


def estimate_tokens(text: str) -> int:
    """토큰 수 추정 (대략 4자 = 1토큰)"""
    return len(text) // 4


def recursive_split(text: str, chunk_size: int = CHUNK_SIZE) -> List[str]:
    """재귀적 텍스트 분할"""
    if estimate_tokens(text) <= chunk_size:
        return [text]
    
    for sep in ["\n## ", "\n### ", "\n\n", "\n• ", "\n", ". ", " "]:
        if sep in text:
            parts = text.split(sep)
            chunks = []
            current = ""
            for part in parts:
                test = current + sep + part if current else part
                if estimate_tokens(test) <= chunk_size:
                    current = test
                else:
                    if current:
                        chunks.append(current)
                    current = part
            if current:
                chunks.append(current)
            return chunks
    
    char_limit = chunk_size * 4
    return [text[i:i+char_limit] for i in range(0, len(text), char_limit)]


def find_images_in_text(text: str) -> List[Dict]:
    """텍스트에서 이미지 마커 찾기 - 개선 버전"""
    pattern = r'\[Image:\s*([^\]]+)\]'
    images = []
    for m in re.finditer(pattern, text):
        path = m.group(1).strip()
        # notion_images로 시작하지 않으면 추가
        if not path.startswith("notion_images/"):
            path = f"notion_images/{path}"
        images.append({
            "path": path,
            "start": m.start(),
            "end": m.end(),
            "marker": m.group(0)
        })
    return images


def encode_image_to_base64(image_path: str) -> Optional[str]:
    """이미지를 base64로 인코딩"""
    try:
        with open(image_path, "rb") as f:
            return base64.b64encode(f.read()).decode("utf-8")
    except Exception:
        return None


def generate_image_description(image_path: str, page_title: str, section_title: str, 
                                text_before: str, text_after: str, client: OpenAI) -> str:
    """이미지 설명 생성"""
    if image_path.startswith("http"):
        try:
            response = httpx.get(image_path, timeout=30)
            base64_image = base64.b64encode(response.content).decode("utf-8")
        except Exception:
            return "[이미지 로드 실패]"
    else:
        base64_image = encode_image_to_base64(image_path)
    
    if not base64_image:
        return "[이미지 로드 실패]"
    
    media_type = "image/png" if ".png" in image_path.lower() else "image/jpeg"
    
    prompt = f"""이미지를 검색에 유용하게 3~5문장으로 설명하세요.
맥락: {page_title} > {section_title}
앞: {text_before[:100]}
뒤: {text_after[:100]}"""

    try:
        response = client.chat.completions.create(
            model=IMAGE_MODEL,
            messages=[{"role": "user", "content": [
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": f"data:{media_type};base64,{base64_image}"}}
            ]}],
            max_tokens=300
        )
        return response.choices[0].message.content.strip()
    except Exception:
        return "[이미지 설명 생성 실패]"


def process_page_data(page_data: Dict, openai_client: OpenAI, image_client: OpenAI = None) -> List[Chunk]:
    """페이지 데이터를 청크로 변환"""
    page_id = page_data["page_id"]
    content = page_data["content"]
    page_title = page_data.get("title", "Untitled")
    
    if not content.strip():
        return []
    
    sections = split_by_markdown_headers(content, page_id, page_title)
    chunks = []
    
    for sec_idx, section in enumerate(sections):
        split_texts = recursive_split(section["full_text"]) if estimate_tokens(section["full_text"]) > CHUNK_SIZE else [section["full_text"]]
        
        for chunk_idx, chunk_text in enumerate(split_texts):
            chunk_id = f"{page_id}_{sec_idx}_{chunk_idx}"
            chunk_images = find_images_in_text(chunk_text)
            image_paths = [img["path"] for img in chunk_images]
            image_descriptions = []
            
            if chunk_images and image_client:
                for img in chunk_images:
                    start, end = img["start"], img["end"]
                    text_before = chunk_text[max(0, start-200):start]
                    text_after = chunk_text[end:end+200]
                    desc = generate_image_description(img["path"], page_title, section["title"], text_before, text_after, image_client)
                    image_descriptions.append(desc)
            
            combined_text = chunk_text
            for img, desc in zip(chunk_images, image_descriptions):
                combined_text = combined_text.replace(img["marker"], f"[이미지: {desc}]")
            
            chunks.append(Chunk(
                chunk_id=chunk_id, page_id=page_id, text=chunk_text, combined_text=combined_text,
                has_image=len(chunk_images) > 0, image_paths=image_paths, image_descriptions=image_descriptions,
                page_title=page_title, section_title=section["title"], section_path=section["path"],
                heading_level=section["level"], chunk_index=chunk_idx,
                created_time=page_data.get("created_time", ""),
                last_edited_time=page_data.get("last_edited_time", ""),
                properties=page_data.get("properties", {})
            ))
    
    return chunks


# ========== 임베딩 & Qdrant ==========
def generate_embeddings(chunks: List[Chunk], client: OpenAI) -> List[List[float]]:
    """청크들의 임베딩 생성"""
    texts = [c.combined_text for c in chunks]
    print(f"\n🔢 임베딩 생성 중... ({len(texts)}개)")
    
    all_embeddings = []
    for i in range(0, len(texts), 100):
        batch = texts[i:i+100]
        response = client.embeddings.create(model=EMBEDDING_MODEL, input=batch)
        all_embeddings.extend([d.embedding for d in response.data])
        print(f"  → {min(i+100, len(texts))}/{len(texts)}")
    
    print(f"  → 완료 (차원: {len(all_embeddings[0])})")
    return all_embeddings


def init_qdrant(client: QdrantClient, recreate: bool = False):
    """Qdrant 컬렉션 초기화"""
    exists = any(c.name == QDRANT_COLLECTION for c in client.get_collections().collections)
    
    if exists and recreate:
        client.delete_collection(QDRANT_COLLECTION)
        exists = False
    
    if not exists:
        client.create_collection(
            collection_name=QDRANT_COLLECTION,
            vectors_config=VectorParams(size=EMBEDDING_DIM, distance=Distance.COSINE)
        )
        for field, ftype in [("page_id", PayloadSchemaType.KEYWORD), ("page_title", PayloadSchemaType.KEYWORD)]:
            client.create_payload_index(collection_name=QDRANT_COLLECTION, field_name=field, field_schema=ftype)
        print(f"✅ 컬렉션 생성: {QDRANT_COLLECTION}")


def store_to_qdrant(chunks: List[Chunk], embeddings: List[List[float]], client: QdrantClient):
    """Qdrant에 청크 저장 - 이미지 정보 포함"""
    points = []
    for chunk, emb in zip(chunks, embeddings):
        pid = hashlib.md5(chunk.chunk_id.encode()).hexdigest()[:32]
        pid = f"{pid[:8]}-{pid[8:12]}-{pid[12:16]}-{pid[16:20]}-{pid[20:]}"
        
        props = {k: (v if isinstance(v, (str, int, float, bool, list)) else str(v)) 
                for k, v in chunk.properties.items()}
        
        points.append(PointStruct(id=pid, vector=emb, payload={
            "chunk_id": chunk.chunk_id,
            "page_id": chunk.page_id,
            "text": chunk.text,
            "combined_text": chunk.combined_text,
            "has_image": chunk.has_image,
            "image_paths": chunk.image_paths,  # ✅ 이미지 경로들
            "image_descriptions": chunk.image_descriptions,  # ✅ 이미지 설명들
            "page_title": chunk.page_title,
            "section_title": chunk.section_title,
            "section_path": chunk.section_path,
            "properties": props
        }))
    
    for i in range(0, len(points), 100):
        client.upsert(collection_name=QDRANT_COLLECTION, points=points[i:i+100])
    
    print(f"✅ {len(points)}개 청크 저장 완료")


# ========== 검색 함수 개선 ==========
def search_with_images(query: str, openai_client: OpenAI, qdrant_client: QdrantClient, limit: int = 10) -> List[Dict]:
    """이미지 정보를 포함한 벡터 검색"""
    response = openai_client.embeddings.create(model=EMBEDDING_MODEL, input=[query])
    results = qdrant_client.query_points(
        collection_name=QDRANT_COLLECTION, 
        query=response.data[0].embedding, 
        limit=limit,
        with_payload=True
    )
    
    return [{
        "score": h.score,
        "page_title": h.payload["page_title"],
        "section_title": h.payload["section_title"],
        "text": h.payload["text"],
        "combined_text": h.payload.get("combined_text", ""),
        "has_image": h.payload.get("has_image", False),
        "image_paths": h.payload.get("image_paths", []),
        "image_descriptions": h.payload.get("image_descriptions", [])
    } for h in results.points]


# ========== 데이터 저장/로드 ==========
def save_data(data: List[Dict], filepath: str = "notion_data.json"):
    """데이터를 JSON 파일로 저장"""
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"💾 저장: {filepath}")


def load_data(filepath: str = "notion_data.json") -> List[Dict]:
    """JSON 파일에서 데이터 로드"""
    with open(filepath, "r", encoding="utf-8") as f:
        return json.load(f)






In [33]:
import sys

data_file = "notion_data.json"

# 설정
force_recreate = False  # True: 전체 재수집, False: 증분 수집
check_updates = True    # 수정된 페이지도 확인할지 여부
skip_fetch = False      # True: 데이터 수집 건너뛰기

print("🚀 Notion RAG Pipeline (스마트 수집 모드)\n" + "=" * 60)

# 환경변수 체크
if not skip_fetch:
    if not NOTION_TOKEN:
        print("❌ NOTION_TOKEN 환경변수가 필요합니다!")
        sys.exit(1)
    if not DATA_SOURCE_ID:
        print("❌ DATA_SOURCE_ID 환경변수가 필요합니다!")
        sys.exit(1)

if not OPENROUTER_API_KEY:
    print("❌ OPENROUTER_API_KEY 환경변수가 필요합니다!")
    sys.exit(1)

# 1️⃣ 데이터 수집 (스마트 모드)
if skip_fetch:
    print("📂 저장된 데이터 로드...")
    all_data = load_data(data_file)
else:
    # 스마트 수집: 새 페이지만 추가, 수정된 페이지 업데이트
    collector = NotionDataSourceCollector(NOTION_TOKEN, DATA_SOURCE_ID)
    
    existing_info = check_existing_data(data_file)
    
    if existing_info["exists"] and not force_recreate:
        print(f"✅ 기존 데이터 발견: {existing_info['count']}개 페이지")
        
        # 새 페이지 수집
        new_data = collect_missing_pages(collector, existing_info["page_ids"], data_file)
        
        # 수정된 페이지 업데이트
        if check_updates:
            all_data = load_data(data_file)
            all_data = update_changed_pages(collector, all_data, data_file)
        else:
            all_data = load_data(data_file)
        
        if not new_data:
            print("\n✨ 모든 데이터가 최신 상태입니다!")
    else:
        # 초기 수집 또는 전체 재수집
        print("📥 전체 데이터 수집 시작...")
        all_data = collector.collect_all()  # limit 제거 (전체 수집)
        save_data(all_data, data_file)

if not all_data:
    print("⚠️ 데이터 없음")
    sys.exit(0)

# 2️⃣ 클라이언트 초기화
openai_client = OpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url=OPENROUTER_BASE_URL
)

image_client = (
    OpenAI(api_key=AZURE_AI_CREDENTIAL, base_url=AZURE_AI_ENDPOINT)
    if AZURE_AI_CREDENTIAL else None
)

qdrant_client = QdrantClient(path="./qdrant_data")

# 3️⃣ 스마트 인덱싱
print("\n" + "=" * 60)
print("🔄 Qdrant 인덱싱")
print("=" * 60)

qdrant_info = check_qdrant_data(qdrant_client)
data_info = check_existing_data(data_file)

# 새로 인덱싱할 페이지 확인
if not force_recreate and qdrant_info["exists"]:
    new_page_ids = data_info["page_ids"] - qdrant_info["page_ids"]
    
    if not new_page_ids:
        print("✅ 모든 데이터가 이미 인덱싱되어 있습니다!")
        pages_to_index = []
    else:
        print(f"\n📊 {len(new_page_ids)}개 새 페이지를 인덱싱합니다.")
        pages_to_index = [d for d in all_data if d["page_id"] in new_page_ids]
else:
    # 전체 재인덱싱
    print("\n♻️ 전체 재인덱싱 시작...")
    init_qdrant(qdrant_client, recreate=True)
    pages_to_index = all_data

# 4️⃣ 청킹 및 임베딩 (필요한 페이지만)
if pages_to_index:
    print(f"\n📝 {len(pages_to_index)}개 페이지 청킹 중...")
    all_chunks = []
    
    for page in pages_to_index:
        chunks = process_page_data(page, openai_client, image_client)
        all_chunks.extend(chunks)
        print(f"  {page.get('title', 'Untitled')}: {len(chunks)}개 청크")
    
    print(f"\n📊 총 {len(all_chunks)}개 청크 생성")
    
    if all_chunks:
        embeddings = generate_embeddings(all_chunks, openai_client)
        store_to_qdrant(all_chunks, embeddings, qdrant_client)
    else:
        print("⚠️ 생성된 청크 없음")

# 5️⃣ 테스트 검색
print("\n" + "=" * 60)
print("🧪 검색 테스트")
print("=" * 60)

test_queries = ["프로젝트 목표", "데이터 처리"]

for q in test_queries:
    print(f"\n🔍 쿼리: '{q}'")
    results = search_with_images(q, openai_client, qdrant_client, limit=3)
    
    for i, r in enumerate(results, 1):
        print(f"\n  [{i}] {r['page_title']} > {r['section_title']}")
        print(f"      점수: {r['score']:.4f}")
        print(f"      이미지: {len(r['image_paths'])}개")
        print(f"      내용: {r['text'][:100]}...")

print("\n" + "=" * 60)
print("🎉 완료!")
print("=" * 60)

🚀 Notion RAG Pipeline (스마트 수집 모드)
✅ 기존 데이터: 10개 페이지
   마지막 업데이트: 2025-12-14T08:40:00.000Z
✅ 기존 데이터 발견: 10개 페이지

🔍 Notion에서 전체 페이지 목록 가져오는 중...
  📄 89개 페이지 로드됨...

📊 분석 결과:
   전체 페이지: 89개
   기존 페이지: 10개
   새 페이지: 79개

🚀 79개 새 페이지 수집 시작...

[1/79] 📄 동영상 추천 기술 조사


KeyboardInterrupt: 